In [ ]:
import keras
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
from keras.callbacks import EarlyStopping

# import keras.backend as K

In [ ]:
df = pd.read_csv("data.csv")
df.drop(columns=['Model', 'Market Category', 'Number of Doors'], inplace=True)
df

FileNotFoundError: ignored

In [ ]:
df_disc = df.copy()
df_disc

In [ ]:
for col in df_disc.columns:
    df_disc = df_disc[df_disc[col].notnull()]

In [ ]:
df_disc['MSRP'] = pd.qcut(df_disc['MSRP'], 2, labels=['cheap', 'expensive'])
df_disc['MSRP'].replace({'cheap': 0, 'expensive': 1}, inplace=True)
df_disc.head()

In [ ]:
string_col = ['Make', 'Engine Fuel Type', 'Engine HP', 'Transmission Type', 'Driven_Wheels', 'Vehicle Size', 'Vehicle Style']
for col in string_col:
    df_disc[col] = df_disc[col].astype('category')
    df_disc[col] = df_disc[col].cat.codes

df_disc

In [ ]:
X = df_disc.drop(columns=['MSRP'])
y = df_disc.MSRP # Target variable

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [ ]:
X_train.shape

In [ ]:
# def get_f1(y_true, y_pred): #taken from old keras source code
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     recall = true_positives / (possible_positives + K.epsilon())
#     f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
#     return f1_val

In [ ]:
model=keras.models.Sequential([
    keras.layers.Dense(1024, input_dim = X_train.shape[1], activation='relu'),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=64, activation='relu'),
    
    keras.layers.Dense(units=1, activation="sigmoid"),
],name="Initial_model",)
model.summary()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
learning_rate = 0.001
optimizer = keras.optimizers.Adam(lr=learning_rate)

In [ ]:
model.compile(optimizer=optimizer,
            loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=1000,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=1000, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1, callbacks=[early_stopping_monitor])

Epoch 1/1000
11/11 [==============================] - 1s 90ms/step - loss: 72.8909 - accuracy: 0.5103 - val_loss: 19.0245 - val_accuracy: 0.4992
Epoch 2/1000
11/11 [==============================] - 0s 45ms/step - loss: 12.6774 - accuracy: 0.5331 - val_loss: 7.9436 - val_accuracy: 0.5635
Epoch 3/1000
11/11 [==============================] - 1s 46ms/step - loss: 6.7194 - accuracy: 0.5766 - val_loss: 1.4846 - val_accuracy: 0.7986
Epoch 4/1000
11/11 [==============================] - 0s 44ms/step - loss: 2.9089 - accuracy: 0.7123 - val_loss: 3.0456 - val_accuracy: 0.6413
Epoch 5/1000
11/11 [==============================] - 1s 46ms/step - loss: 1.8947 - accuracy: 0.7540 - val_loss: 1.8815 - val_accuracy: 0.8164
Epoch 6/1000
11/11 [==============================] - 1s 46ms/step - loss: 1.2094 - accuracy: 0.8195 - val_loss: 0.8266 - val_accuracy: 0.8393
Epoch 7/1000
11/11 [==============================] - 1s 46ms/step - loss: 0.8698 - accuracy: 0.8320 - val_loss: 0.9121 - val_accuracy: 0.8

In [ ]:
_, train_acc = model.evaluate(X_train, y_train, verbose=0)
_, test_acc = model.evaluate(X_test, y_test, verbose=0)

In [ ]:
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))